In [1]:
import numpy as np

np.show_config()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /opt/arm64-builds/include
    lib directory: /opt/arm64-builds/lib
    name: openblas64
    openblas configuration: USE_64BITINT=1 DYNAMIC_ARCH=1 DYNAMIC_OLDER= NO_CBLAS=
      NO_LAPACK= NO_LAPACKE= NO_AFFINITY=1 USE_OPENMP= SANDYBRIDGE MAX_THREADS=3
    pc file directory: /usr/local/lib/pkgconfig
    version: 0.3.23.dev
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep4350285776
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: cc
    linker: ld64
    linker args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    name: clang
    version: 14.0.0
  c++:
    args: -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: c++
    linker: ld64
    linker

# side model

In [2]:
import pandas as pd

df_features = pd.read_parquet("data/features_25m.parquet")
label_side = np.load("data/label_side_25m.npy")
print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(106174, 11841)
(106174, 7)


2h_williams_r_ddt_lag39               808
2h_evenbetter_sinewave_short_lag28    808
2h_evenbetter_sinewave_short_lag20    808
2h_evenbetter_sinewave_short_lag21    808
2h_evenbetter_sinewave_short_lag22    808
                                     ... 
25m_comb_spectrum_pwr_9                 0
25m_comb_spectrum_pwr_8                 0
25m_comb_spectrum_pwr_7                 0
25m_comb_spectrum_pwr_6                 0
25m_ac_37                               0
Length: 11841, dtype: int64

In [3]:
df_features = df_features[808:]
label_side = label_side[808:]

print(df_features.shape)
print(label_side.shape)

label_side

(105366, 11841)
(105366, 7)


array([[ 1.58600034e+12,  6.82226000e+03,  6.70108000e+03, ...,
         6.62999000e+03,  3.30846160e+04, -1.00000000e+00],
       [ 1.58600388e+12,  6.70102000e+03,  6.73412000e+03, ...,
         6.65787000e+03,  3.11395250e+04,  1.00000000e+00],
       [ 1.58601396e+12,  6.73412000e+03,  6.76520000e+03, ...,
         6.70833000e+03,  3.04400920e+04,  1.00000000e+00],
       ...,
       [ 1.73827176e+12,  1.04961400e+05,  1.05061000e+05, ...,
         1.04864200e+05,  2.00223200e+03, -1.00000000e+00],
       [ 1.73827518e+12,  1.05060900e+05,  1.05209100e+05, ...,
         1.04997700e+05,  1.94644700e+03, -1.00000000e+00],
       [ 1.73827968e+12,  1.05209100e+05,  1.04676900e+05, ...,
         1.04628300e+05,  1.96454500e+03, -1.00000000e+00]])

In [4]:
from custom_indicators.config import SIDE_ALL

side_features = df_features[SIDE_ALL]
print(side_features.shape)
side_label = (label_side[:, 6].astype(int) == 1).astype(int)
print(np.unique(side_label, return_counts=True))

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features[:train_test_split_point]
side_features_test = side_features[train_test_split_point:]
side_label_train = side_label[:train_test_split_point]
side_label_test = side_label[train_test_split_point:]

print(np.unique(side_label_test, return_counts=True))

(105366, 2368)
(array([0, 1]), array([50766, 54600]))
(array([0, 1]), array([10135, 10939]))


In [5]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        # "early_stopping_round": 100,
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 100),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        # valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-08 18:28:04,923] A new study created in memory with name: no-name-be067263-6c6f-49e8-9a86-d7f032ca327e
[I 2025-03-08 18:29:17,241] Trial 0 finished with value: 0.8612488422477196 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 33, 'max_depth': 40, 'min_gain_to_split': 0.2526514275323875, 'min_data_in_leaf': 63, 'lambda_l1': 24.875827136430892, 'lambda_l2': 67.20378396623155, 'num_boost_round': 600}. Best is trial 0 with value: 0.8612488422477196.


precision: 0.7807847176701939, recall: 0.7622268945973124, f1: 0.7713942085299288


[I 2025-03-08 18:29:31,522] Trial 1 finished with value: 0.867576735011615 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 166, 'max_depth': 45, 'min_gain_to_split': 0.19195340084362575, 'min_data_in_leaf': 259, 'lambda_l1': 27.312295042019244, 'lambda_l2': 26.412747172508546, 'num_boost_round': 124}. Best is trial 1 with value: 0.867576735011615.


precision: 0.7805547926393848, recall: 0.779413109059329, f1: 0.7799835330710823


[I 2025-03-08 18:30:07,503] Trial 2 finished with value: 0.8691634594010206 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 178, 'max_depth': 44, 'min_gain_to_split': 0.3453876651682137, 'min_data_in_leaf': 237, 'lambda_l1': 43.5705879879008, 'lambda_l2': 73.48561669299627, 'num_boost_round': 383}. Best is trial 2 with value: 0.8691634594010206.


...

precision: 0.7799854174261757, recall: 0.7823384221592468, f1: 0.7811601478709324


[I 2025-03-08 20:42:57,021] Trial 98 finished with value: 0.8655690278326422 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 206, 'max_depth': 45, 'min_gain_to_split': 0.3428795644322034, 'min_data_in_leaf': 81, 'lambda_l1': 37.24769503144298, 'lambda_l2': 81.99211526611784, 'num_boost_round': 424}. Best is trial 85 with value: 0.8693945926897028.


precision: 0.7801684673136788, recall: 0.7789560288874668, f1: 0.7795617766799323


[I 2025-03-08 20:44:51,218] Trial 99 finished with value: 0.8688101885177222 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 170, 'max_depth': 51, 'min_gain_to_split': 0.26706714715722557, 'min_data_in_leaf': 299, 'lambda_l1': 13.968674668710008, 'lambda_l2': 72.36362003248024, 'num_boost_round': 401}. Best is trial 85 with value: 0.8693945926897028.


precision: 0.7799108361386589, recall: 0.7836182466404608, f1: 0.7817601459188327


In [6]:
study.best_params

{'extra_trees': True,
 'boosting': 'dart',
 'num_leaves': 32,
 'max_depth': 50,
 'min_gain_to_split': 0.3224598396344323,
 'min_data_in_leaf': 80,
 'lambda_l1': 11.077546467647013,
 'lambda_l2': 85.8427512785022,
 'num_boost_round': 379}

In [7]:
params = {
    # "is_unbalance": True,
    **study.best_params,
}

dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(params, dtrain)

In [8]:
import plotly.express as px

res = model.predict(side_features_train)

fig = px.histogram(res, nbins=100)
fig.show()

In [9]:
import plotly.express as px

res = model.predict(side_features_test)
print(roc_auc_score(side_label_test, res))

fig = px.histogram(res, nbins=100)
fig.show()

0.8688723712647338


In [10]:
model.save_model("custom_indicators/models/model_side.txt")

# meta model

In [15]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from custom_indicators.config import SIDE_ALL

df_features = pd.read_parquet("data/features_25m.parquet")
meta_label = np.load("data/label_meta_25m.npy")

model_side = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
side_model_res = model_side.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_res

df_features.isna().sum(axis=0).sort_values(ascending=False)

2h_voss_ddt_lag9                   808
2h_adaptive_stochastic_ddt_lag5    808
2h_adaptive_stochastic_ddt_lag3    808
2h_adaptive_stochastic_ddt_lag2    808
2h_adaptive_stochastic_ddt_lag1    808
                                  ... 
25m_comb_spectrum_pwr_14             0
25m_comb_spectrum_pwr_13             0
25m_comb_spectrum_pwr_12             0
25m_comb_spectrum_pwr_11             0
side_model_res                       0
Length: 11842, dtype: int64

In [16]:
df_features = df_features[808:]
meta_label = meta_label[808:]

print(df_features.shape)

assert df_features.shape[0] == meta_label.shape[0]
print(meta_label.shape)

(105366, 11842)
(105366, 7)


In [17]:
from custom_indicators.config import META_ALL

meta_features = df_features[META_ALL]
print(meta_features.shape)
meta_label = meta_label[:, 6].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features[:train_test_split_point]
meta_features_test = meta_features[train_test_split_point:]
meta_label_train = meta_label[:train_test_split_point]
meta_label_test = meta_label[train_test_split_point:]

np.unique(meta_label_test, return_counts=True)

(105366, 2368)


(array([0, 1]), array([ 4740, 16334]))

In [19]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 100),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    auc = roc_auc_score(meta_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}")
    return f1


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

[I 2025-03-09 00:39:46,143] A new study created in memory with name: no-name-cc442e5d-1385-4384-9b36-c30187edd3b6
[I 2025-03-09 00:40:46,500] Trial 0 finished with value: 0.8253643410852713 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 211, 'max_depth': 58, 'min_gain_to_split': 0.2209742462937076, 'min_data_in_leaf': 115, 'lambda_l1': 22.654427355888263, 'lambda_l2': 10.63943093000907, 'num_boost_round': 262}. Best is trial 0 with value: 0.8253643410852713.


precision: 0.8362025634581554, recall: 0.8148034774090853, f1: 0.8253643410852713, auc: 0.7604377294855957


[I 2025-03-09 00:42:05,340] Trial 1 finished with value: 0.7487705947808607 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 189, 'max_depth': 36, 'min_gain_to_split': 0.24431397988001136, 'min_data_in_leaf': 103, 'lambda_l1': 92.33619278791869, 'lambda_l2': 2.108174289446243, 'num_boost_round': 535}. Best is trial 0 with value: 0.8253643410852713.


precision: 0.9049891540130152, recall: 0.638545365495286, f1: 0.7487705947808607, auc: 0.7677831026271725


[I 2025-03-09 00:43:14,698] Trial 2 finished with value: 0.7603645777919636 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 45, 'max_depth': 92, 'min_gain_to_split': 0.26519168284752215, 'min_data_in_leaf': 205, 'lambda_l1': 7.046217306068517, 'lambda_l2': 6.031167550422434, 'num_boost_round': 671}. Best is trial 0 with value: 0.8253643410852713.


...

precision: 0.827551081550722, recall: 0.8455369168605363, f1: 0.836447324591951, auc: 0.7618946449615335


[I 2025-03-09 07:47:35,157] Trial 98 finished with value: 0.8563759922334533 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 227, 'max_depth': 96, 'min_gain_to_split': 0.027501967043091026, 'min_data_in_leaf': 179, 'lambda_l1': 0.0044415949969807515, 'lambda_l2': 1.9453925601760353, 'num_boost_round': 812}. Best is trial 98 with value: 0.8563759922334533.


precision: 0.8024400684931506, recall: 0.9180849761234235, f1: 0.8563759922334533, auc: 0.7600784700598633


[I 2025-03-09 07:51:03,300] Trial 99 finished with value: 0.8556810061454909 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 225, 'max_depth': 95, 'min_gain_to_split': 0.030261869122553776, 'min_data_in_leaf': 183, 'lambda_l1': 0.0074683325057127575, 'lambda_l2': 1.092236140255359, 'num_boost_round': 925}. Best is trial 98 with value: 0.8563759922334533.


precision: 0.8025306954050722, recall: 0.9163707603771275, f1: 0.8556810061454909, auc: 0.759422852283477


In [20]:
study.best_params

{'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 227,
 'max_depth': 96,
 'min_gain_to_split': 0.027501967043091026,
 'min_data_in_leaf': 179,
 'lambda_l1': 0.0044415949969807515,
 'lambda_l2': 1.9453925601760353,
 'num_boost_round': 812}

In [21]:
params = {
    "is_unbalance": True,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

In [22]:
import plotly.express as px

res = model.predict(meta_features_test)

fig = px.histogram(res, nbins=100)
fig.show()

In [23]:
model.save_model("custom_indicators/models/model_meta.txt")